In [2]:
#Speaker ID Array to to be considered Task 1#
speaker_id=["84", "174", "251", "422", "652", "777", "1462", "1673", "1919", "1988"]

In [3]:
#All Imports#
import numpy as np
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#All Variable and Static Values Initialization#
speaker_info=[]
extracted_features_seg=[]
extracted_features_frame=[]
speakers_ground_truth = []
gender_ground_truth = []
speakers_ground_truth_seg = []
gender_ground_truth_seg = []
speaker_feature_numbers=[]
segment_length_ms=2000
sr=16000
segment_length=int(sr*segment_length_ms/1000)
audio_lengths=[]
print("seglength",segment_length)
metadata_file_path="/Users/maitry/Data/Project_LibriSpeech/LibriSpeech/SPEAKERS.TXT"
audiodata_file_path="/Users/maitry/Data/Project_LibriSpeech/LibriSpeech/dev-clean/"
pickle_files_path="/Users/maitry/Data/Project_LibriSpeech/LibriSpeech/pickle/"





seglength 32000


In [5]:
#Speaker data loading, only first 12 speakers are loaded for easy analysis#
count=0
with open(metadata_file_path) as o:

    metadata=o.read().split("\n")[12:-1]
    #print(metadata)
    for row in metadata:
        row=row.replace(" ","")
        row=row.strip().split("|")
        #print(row[0])
        
        if (row[0]) in speaker_id and count <10:
            print(row)
            count+=1
            speaker_info.append([row[0],row[1]])
print(len(speaker_info))
speakerwise_segment_count = np.zeros((10, 1))

speaker_info = np.concatenate((np.array(speaker_info), speakerwise_segment_count), axis=1)

print(speaker_info)

['84', 'F', 'dev-clean', '8.02', 'ChristieNowak']
['174', 'M', 'dev-clean', '8.04', 'PeterEastman']
['251', 'M', 'dev-clean', '8.04', 'MarkNelson']
['422', 'M', 'dev-clean', '8.38', 'PresidentLethe']
['652', 'M', 'dev-clean', '8.31', 'ScottWalter']
['777', 'M', 'dev-clean', '8.06', 'fling93']
['1462', 'F', 'dev-clean', '8.04', 'E.Tavano']
['1673', 'F', 'dev-clean', '8.07', 'Tonia']
['1919', 'F', 'dev-clean', '8.17', 'nprigoda']
['1988', 'F', 'dev-clean', '8.16', 'Ransom']
10
[['84' 'F' '0.0']
 ['174' 'M' '0.0']
 ['251' 'M' '0.0']
 ['422' 'M' '0.0']
 ['652' 'M' '0.0']
 ['777' 'M' '0.0']
 ['1462' 'F' '0.0']
 ['1673' 'F' '0.0']
 ['1919' 'F' '0.0']
 ['1988' 'F' '0.0']]


In [6]:

def fixed_length_segmentation(audio, segment_length):
        segments=[]
        count=0
        i=count+segment_length
        end=len(audio)
        while i < end:
            segment=np.array(audio[count:i])
            count=i
            i+=segment_length
            
            segments.append(segment)
           # print("***",len(segment))
        segment=np.array(audio[i-segment_length:])
        segment = np.pad(segment, (0, segment_length - len(segment)), mode='constant')
        #print("##",len(segment))
        segments.append(segment)
    
        return (segments,  len(segments))

In [8]:
#Feature extraction from audio files, segmentwise and framewise, of all the speakers , store frame and segment count per speaker#
segmentwise_audio=np.empty((0, 32000))

for i in range(0, 10):
    #print(speaker_id[i])
    
    current_speaker_path=audiodata_file_path+str(speaker_info[i][0])
    #print(current_speaker_path)
    if os.path.isdir(current_speaker_path):
        chapters= os.listdir(current_speaker_path)
        #print("files in {current_speaker_path}:")
        count=0
        for chapter in chapters:
            file_path=current_speaker_path+"/"+chapter    
            #print(file_path)
            if os.path.isdir(file_path):
                files=os.listdir(file_path)
                for file_name in files:
                    if file_name.endswith(".flac"):
                            y, sr = librosa.load(file_path+"/"+file_name,sr=16000)
                            segments, seg_count=fixed_length_segmentation(y,segment_length)
                            #segmentwise_audio=np.concatenate((segmentwise_audio,segments),axis=0)
                            for seg in segments:
                                extracted_features_seg.append([seg,speaker_info[i][0],speaker_info[i][1]])
            
                            count+=seg_count
 #                           print(len(segments[0]))
#                            print(len(segments[1]))
    print(count)                           
    speaker_info[i][2]=count   

                            



        


275
276
276
270
281
285
297
262
275
284


In [9]:
import pickle
import pandas as pd
extracted_features_seg
def pickle_file_store(file_path,file_name,data):
    with open(file_path+file_name, 'wb') as file:    
    # A new file will be created
        pickle.dump(data, file)

extracted_features_seg_df=pd.DataFrame(extracted_features_seg,columns=["Audio","Speaker","Gender"])
segment_features_file_name="segmentwise_audio_2seconds.pkl"
pickle_file_store(pickle_files_path,segment_features_file_name,extracted_features_seg_df)


In [10]:
#load stored features#
def pickle_file_load(file_path,file_name):
    with (open(file_path+file_name, "rb")) as testfile:
        file_new=pickle.load(testfile)
    return file_new
extracted_features_segwise=pickle_file_load(pickle_files_path,segment_features_file_name)


In [ ]:
import fairseq
import torch
model_path= "/Users/maitry/Data/Project_LibriSpeech/LibriSpeech/wav2vec_small.pt"
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([model_path])
model = model[0]
model.eval()
new_extracted_features_segwise=[]
for index, row in extracted_features_segwise.iterrows():
    root_row=row
    row=np.array(row["Audio"])
    row=row.reshape(-1,1)
    #print(row.shape)
    row=row.T
    current=torch.tensor(row)
    #print(current.shape)
    z = model.feature_extractor(current)
    #c = model.feature_aggregator(z)
    print(z[0].shape)
    
    new_extracted_features_segwise.append([z[0], root_row["Audio"], root_row["Speaker"],root_row["Gender"]])
    
   # print(c.shape)
extracted_features_seg_df=pd.DataFrame(extracted_features_seg,columns=["features","Audio","Speaker","Gender"])
segment_features_audio_file_name="segmentwise_features_2seconds.pkl"
#pickle_file_store(pickle_files_path,segment_features_audio_file_name,extracted_features_seg_df)


/Users/maitry/Library/Python/3.9/lib/python/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size([512, 99])
torch.Size

In [ ]:
#qualitative analysis using predictive models#
def one_hot_encode(strings):
    # Step 1: Find unique elements
    unique_elements = sorted(set(strings))
    
    # Step 2: Assign unique indices to each category
    index_map = {element: index for index, element in enumerate(unique_elements)}
    
    # Step 3: Create one-hot encoding
    num_categories = len(unique_elements)
    one_hot_encoded = []
    for string in strings:
        one_hot_vector = [0] * num_categories
        index = index_map[string]
        one_hot_vector[index] = 1
        one_hot_encoded.append(one_hot_vector)
    
    return one_hot_encoded

y_all=one_hot_encode()


In [23]:
import torch
wav_input_16khz = torch.randn(1,10000)
print(wav_input_16khz.shape)

torch.Size([1, 10000])


In [1]:


# Example usage
random_strings = ["a", "b", "c", "d", "a", "f"]
output = one_hot_encode(random_strings)
print("Output:")
print(output)

Output:
[[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0, 1]]
